In [3]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from math import *
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor,AdaBoostRegressor, RandomForestClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.tree import DecisionTreeRegressor
import seaborn as sns
import time

In [4]:
train_df = pd.read_csv('../data/raw/dmml1_train.csv')
test_df = pd.read_csv('../data/raw/dmml1_test.csv')
store = pd.read_csv('../data/raw/dmml1_stores.csv')

In [5]:
train_df

,Store ID,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,4,2015-04-30,6884,716,1,1,0,0
1,1,3,2015-04-29,6764,756,1,1,0,0
2,1,2,2015-04-28,6861,678,1,1,0,0
3,1,1,2015-04-27,6523,647,1,1,0,0
4,1,7,2015-04-26,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
246898,300,6,2013-01-05,5194,569,1,0,0,0
246899,300,5,2013-01-04,5524,646,1,0,0,1
246900,300,4,2013-01-03,5563,718,1,0,0,1
246901,300,3,2013-01-02,6218,753,1,0,0,1


In [6]:
train_df = pd.merge(train_df,store,how="inner",on="Store ID")
test_df = pd.merge(test_df,store,how="inner",on="Store ID")

In [7]:
train_df["CompetitionDistance"].replace(np.nan,train_df["CompetitionDistance"].mean(),inplace=True)
train_df.fillna(0,inplace=True)

In [8]:
train_df["StateHoliday"].replace("0",0,inplace=True)

In [9]:
train_df['Date'] = pd.to_datetime(train_df['Date'])
train_df['Year'] = train_df['Date'].dt.year
train_df['Month'] = train_df['Date'].dt.month
train_df['Day'] = train_df['Date'].dt.day
train_df['WeekOfYear'] = train_df['Date'].dt.isocalendar().week
train_df['Weekend'] = np.where(train_df['DayOfWeek'].isin([6, 7]), 1, 0)  # Samstag = 6, Sonntag = 7

train_df['Quarter'] = train_df['Date'].dt.quarter
train_df['DayOfYear'] = train_df['Date'].dt.dayofyear
train_df['DayOfMonth'] = train_df['Date'].dt.day

train_df['Season'] = train_df['Month'].apply(lambda month: (month%12 // 3 + 1))
train_df['Season'].replace(to_replace=[1,2,3,4], value=['Winter', 'Frühling','Sommer','Herbst'], inplace=True)

# Entfernen der ursprünglichen Date-Spalte
train_df.drop('Date', axis=1, inplace=True)

In [10]:
train_df["StateHoliday"] = train_df["StateHoliday"].replace("a","Public Holiday").replace("b","Ostern").replace("c","Weihnachten").replace(0,"Kein Feiertag")

In [11]:
train_df["StoreType"] = train_df["StoreType"].replace("a","Type A").replace("b","Type B").replace("c","Type C").replace("d","Type D")

In [12]:
train_df["Assortment"] = train_df["Assortment"].replace("a","Basic").replace("b","Extra").replace("c","Extended")

In [13]:
# train_df["PromoInterval"] = train_df["PromoInterval"].replace(0,"No PromoInterval").replace("Jan,Apr,Jul,Oct","From Jan to Oct").replace("Feb,May,Aug,Nov","From Feb to Nov").replace("Mar,Jun,Sept,Dec","From Mar to Dec")


def myPinterval(x):
    if x=='From Feb to Nov':
        return([0,1,0,0,1,0,0,1,0,0,1,0])
    elif x=='From Jan to Oct':
        return([1,0,0,1,0,0,1,0,0,1,0,0])
    elif x== 'From Mar to Dec':
        return([0,0,1,0,0,1,0,0,1,0,0,1])
    else:
        return(np.repeat(0,12).tolist())

proInt = train_df.PromoInterval.apply(myPinterval).tolist()
proInt = pd.DataFrame(proInt, columns = ['PromoInt'+ str(i) for i in range(1,13)])
train_df = train_df.drop('PromoInterval',1).join(proInt)

/var/folders/jq/xvfn2vg55lsc1wx2_2mrh3640000gn/T/ipykernel_54027/3265133220.py:16: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  train_df = train_df.drop('PromoInterval',1).join(proInt)


In [14]:
# Erstellen Sie eine temporäre Spalte, die 1 ist, wenn der Tag ein Feiertag ist, und 0 sonst
train_df['Temp'] = np.where(train_df['StateHoliday'] != 'Kein Feiertag', 1, 0)

# Erstellen Sie die Spalte "TagVorFeiertag", die 1 ist, wenn der nächste Tag ein Feiertag ist, und 0 sonst
train_df['TagVorFeiertag'] = train_df['Temp'].shift(-1).fillna(0)

# Erstellen Sie die Spalte "TagnachFeiertag", die 1 ist, wenn der vorherige Tag ein Feiertag war, und 0 sonst
train_df['TagnachFeiertag'] = train_df['Temp'].shift(1).fillna(0)

# Entfernen Sie die temporäre Spalte
train_df.drop('Temp', axis=1, inplace=True)

In [15]:
StateHoliday_dumm = pd.get_dummies(train_df["StateHoliday"])
train_df =pd.concat([train_df,StateHoliday_dumm],axis=1)
train_df.drop("StateHoliday",axis=1,inplace=True)

In [16]:
StoreType_dumm = pd.get_dummies(train_df["StoreType"])
train_df =pd.concat([train_df,StoreType_dumm],axis=1)
train_df.drop("StoreType",axis=1,inplace=True)

In [17]:
Assortment_dumm = pd.get_dummies(train_df["Assortment"])
train_df =pd.concat([train_df,Assortment_dumm],axis=1)
train_df.drop("Assortment",axis=1,inplace=True)

In [18]:
Assortment_dumm = pd.get_dummies(train_df["Season"])
train_df =pd.concat([train_df,Assortment_dumm],axis=1)
train_df.drop("Season",axis=1,inplace=True)

In [19]:
# PromoInterval_dumm = pd.get_dummies(train_df["PromoInterval"])
# train_df =pd.concat([train_df,PromoInterval_dumm],axis=1)
# train_df.drop("PromoInterval",axis=1,inplace=True)

In [20]:
X = train_df.drop(["Sales","Customers"],axis=1)
Y = train_df["Sales"]

In [21]:
X

,Store ID,DayOfWeek,Open,Promo,SchoolHoliday,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,...,Type B,Type C,Type D,Basic,Extended,Extra,Frühling,Herbst,Sommer,Winter
0,1,4,1,1,0,570.0,11.0,2007.0,1,13.0,...,0,0,0,1,0,0,1,0,0,0
1,1,3,1,1,0,570.0,11.0,2007.0,1,13.0,...,0,0,0,1,0,0,1,0,0,0
2,1,2,1,1,0,570.0,11.0,2007.0,1,13.0,...,0,0,0,1,0,0,1,0,0,0
3,1,1,1,1,0,570.0,11.0,2007.0,1,13.0,...,0,0,0,1,0,0,1,0,0,0
4,1,7,0,0,0,570.0,11.0,2007.0,1,13.0,...,0,0,0,1,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246898,300,6,1,0,0,9260.0,0.0,0.0,0,0.0,...,0,0,0,0,1,0,0,0,0,1
246899,300,5,1,0,1,9260.0,0.0,0.0,0,0.0,...,0,0,0,0,1,0,0,0,0,1
246900,300,4,1,0,1,9260.0,0.0,0.0,0,0.0,...,0,0,0,0,1,0,0,0,0,1
246901,300,3,1,0,1,9260.0,0.0,0.0,0,0.0,...,0,0,0,0,1,0,0,0,0,1


In [22]:
X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.3)

In [23]:
forest = RandomForestClassifier(random_state=0)
forest.fit(X_train, y_train)
importances = forest.feature_importances_
importances

In [ ]:
models=[LinearRegression(), Ridge(), Lasso(), RandomForestClassifier(random_state=42), DecisionTreeRegressor(random_state=42), GradientBoostingRegressor(random_state=42), RandomForestRegressor(n_estimators=42,n_jobs=-1, random_state=42)]
model_names=["LinearRegression","Ridge","Lasso","RandomForestRegressor","DecisionTreeRegressor","GradientBoostingRegressor"]

MSE=[]
r_2=[]
accuracy = []
RMSE = []
training_time = []
d={}

for index in range (len(models)):
    model=models[index]
    start_time = time.time()
    model.fit(X_train,y_train)
    end_time = time.time()
    y_pre=model.predict(X_test)

    MSE.append(round(mean_squared_error(y_true=y_test,y_pred=y_pre),5))
    r_2.append(r2_score(y_true=y_test,y_pred=y_pre))
    accuracy.append((model.score(X_test,y_test))*100)
    RMSE.append(sqrt(mean_squared_error(y_true=y_test,y_pred=y_pre)))
    training_time.append(end_time - start_time)
    print(f'{model_names[index]} wurde trainiert und benötigte {end_time - start_time} Sekunden')

d=pd.DataFrame({'Modelling Name':model_names,'MSE':MSE,"R_2":r_2,"Accuracy":accuracy,"RMSE":RMSE, "Training Time": training_time})
print("Verwendete Features: ", X_train.columns.tolist())

In [ ]:
d[['Modelling Name', 'R_2', 'RMSE']]

,Modelling Name,R_2,RMSE
0,LinearRegression,0.558668,2660.183625
1,Ridge,0.558673,2660.170845
2,Lasso,0.558571,2660.476927
3,RandomForestRegressor,0.931816,1045.615290
4,DecisionTreeRegressor,0.725656,2097.379471
5,GradientBoostingRegressor,0.961891,781.701635


In [ ]:
d

,Modelling Name,MSE,R_2,Accuracy,RMSE,Training Time
0,LinearRegression,7.076577e+06,0.558668,55.866842,2660.183625,1.204188
1,Ridge,7.076509e+06,0.558673,55.867266,2660.170845,0.132173
2,Lasso,7.078137e+06,0.558571,55.857110,2660.476927,4.913526
3,RandomForestRegressor,1.093311e+06,0.931816,93.181551,1045.615290,1.612458
4,DecisionTreeRegressor,4.399001e+06,0.725656,72.565579,2097.379471,22.744838
5,GradientBoostingRegressor,6.110574e+05,0.961891,96.189133,781.701635,9.873735


In [ ]:
dfg

NameError: name 'dfg' is not defined

In [ ]:
model = RandomForestRegressor(n_estimators=42,n_jobs=-1)
model.fit(X,Y)

RandomForestRegressor(n_estimators=42, n_jobs=-1)

In [ ]:
test_df["Open"].replace(np.nan,1,inplace=True)

In [ ]:
test_df["CompetitionDistance"].replace(np.nan,test_df["CompetitionDistance"].mean(),inplace=True)
test_df.fillna(0,inplace=True)

In [ ]:
test_df["Date"] = pd.to_datetime(test_df["Date"])

In [ ]:
test_df["StateHoliday"].replace("0",0,inplace=True)

In [ ]:
test_df["Day"] = test_df["Date"].dt.day
test_df["Month"] = test_df["Date"].dt.month
test_df["Year"] = test_df["Date"].dt.year
test_df.drop("Date",axis=1,inplace=True)

In [ ]:
test_df["StateHoliday"] = test_df["StateHoliday"].replace("a","Public Holiday").replace("b","Ostern").replace("c","Weihnachten").replace(0,"Kein Feiertag")

In [ ]:
test_df["StoreType"] = test_df["StoreType"].replace("a","Type A").replace("b","Type B").replace("c","Type C").replace("d","Type D")

In [ ]:
test_df["Assortment"] = test_df["Assortment"].replace("a","Basic").replace("b","Extra").replace("c","Extended")

In [ ]:
test_df["PromoInterval"] = test_df["PromoInterval"].replace(0,"No PromoInterval").replace("Jan,Apr,Jul,Oct","From Jan to Oct").replace("Feb,May,Aug,Nov","From Feb to Nov").replace("Mar,Jun,Sept,Dec","From Mar to Dec")

In [ ]:
StateHoliday_dumm = pd.get_dummies(test_df["StateHoliday"])
test_df =pd.concat([test_df,StateHoliday_dumm],axis=1)
test_df.drop("StateHoliday",axis=1,inplace=True)

In [ ]:
StoreType_dumm = pd.get_dummies(test_df["StoreType"])
test_df =pd.concat([test_df,StoreType_dumm],axis=1)
test_df.drop("StoreType",axis=1,inplace=True)

In [ ]:
Assortment_dumm = pd.get_dummies(test_df["Assortment"])
test_df =pd.concat([test_df,Assortment_dumm],axis=1)
test_df.drop("Assortment",axis=1,inplace=True)

In [ ]:
PromoInterval_dumm = pd.get_dummies(test_df["PromoInterval"])
test_df =pd.concat([test_df,PromoInterval_dumm],axis=1)
test_df.drop("PromoInterval",axis=1,inplace=True)

In [ ]:
test_df.head()

,Store ID,DayOfWeek,Open,Promo,SchoolHoliday,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,...,Type B,Type C,Type D,Basic,Extended,Extra,From Feb to Nov,From Jan to Oct,From Mar to Dec,No PromoInterval
0,1,5,1,1,1,570.0,11.0,2007.0,1,13.0,...,0,0,0,1,0,0,0,1,0,0
1,1,4,1,1,1,570.0,11.0,2007.0,1,13.0,...,0,0,0,1,0,0,0,1,0,0
2,1,3,1,1,1,570.0,11.0,2007.0,1,13.0,...,0,0,0,1,0,0,0,1,0,0
3,1,2,1,1,1,570.0,11.0,2007.0,1,13.0,...,0,0,0,1,0,0,0,1,0,0
4,1,1,1,1,1,570.0,11.0,2007.0,1,13.0,...,0,0,0,1,0,0,0,1,0,0


In [ ]:
ids = test_df["Id"]
test_df.drop("Id",axis=1,inplace=True)

KeyError: 'Id'

In [ ]:
test_df["Weihnachten"] = 0
test_df["Ostern"] = 0

In [ ]:
test_df = test_df[['Store', 'DayOfWeek', 'Open', 'Promo', 'SchoolHoliday',
       'CompetitionDistance', 'CompetitionOpenSinceMonth',
       'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek',
       'Promo2SinceYear', 'Day', 'Month', 'Year', 'Weihnachten',
       'Ostern', 'No Holiday', 'Public Holiday', 'Type A', 'Type B',
       'Type C', 'Type D', 'Basic', 'Extended', 'Extra', 'From Feb to Nov',
       'From Jan to Oct', 'From Mar to Dec', 'No PromoInterval']]

In [ ]:
Predictions = model.predict(test_df)

In [ ]:
Prediction = pd.DataFrame({"Id":ids,"Sales":Predictions})

In [ ]:
Prediction.to_csv("submission.csv",index=False)